<a href="https://colab.research.google.com/github/rkaclfdl123/TIL/blob/master/Spanner_single_other_data(%EA%B3%A0%EC%A4%80%EC%88%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -L "https://app.roboflow.com/ds/Lg2UvQlfkr?key=iJ6wCnLj7d" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0    986      0 --:--:-- --:--:-- --:--:--   985
100 7653k  100 7653k    0     0  3840k      0  0:00:01  0:00:01 --:--:-- 17.7M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: train/
   creating: train/images/
 extracting: train/images/H_8204-20-0000_01_166_png.rf.1bd4dc2e7df83d43cdaca052712c134d.jpg  
 extracting: train/images/H_8204-20-0000_01_167_png.rf.e1d524452315ce0c009ac804d4feef39.jpg  
 extracting: train/images/H_8204-20-0000_01_168_png.rf.1fe5fed2af59e065b5f536c002e620f5.jpg  
 extracting: train/images/H_8204-20-0000_01_169_png.rf.2b058828acdcc4c5886f76190feb500a.jpg  
 extracting: train/images/H_8204-20-0000_01_170_png.rf.5f272c331a24e77d97262c6f04c9fc0f.jpg  
 extracting: train/images/H_82

In [2]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 9966, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 9966 (delta 15), reused 18 (delta 9), pack-reused 9928
Receiving objects: 100% (9966/9966), 10.33 MiB | 24.78 MiB/s, done.
Resolving deltas: 100% (6898/6898), done.


In [3]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
     |████████████████████████████████| 596 kB 4.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
%cat /content/dataset/data.yaml

cat: /content/dataset/data.yaml: No such file or directory


In [7]:
%cd /
from glob import glob

img_list = glob('/content/train/images/*.jpg')

print(len(img_list))

/
750


In [8]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

600 150


In [9]:
with open('/content/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [11]:
import yaml

with open('/content/train/data.yaml', 'r') as f:
  data = yaml.load(f.read(), Loader=yaml.FullLoader)

print(data)

data['train'] = '/content/train.txt'
data['val'] = '/content/val.txt'

with open('/content/train/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['Spanner']}
{'train': '/content/train.txt', 'val': '/content/val.txt', 'nc': 1, 'names': ['Spanner']}


In [12]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/train/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name spanner_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/train/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=spanner_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0

In [13]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/

<IPython.core.display.Javascript object>

In [16]:
from IPython.display import Image
import os

val_img_path = val_img_list[9]

!python detect.py --weights /content/yolov5/runs/train/spanner_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"

detect: weights=['/content/yolov5/runs/train/spanner_yolov5s_results/weights/best.pt'], source=/content/train/images/H_8204-20-0000_10_537_png.rf.469a0bce2ad29a47a051ee9ed9054512.jpg, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/train/images/H_8204-20-0000_10_537_png.rf.469a0bce2ad29a47a051ee9ed9054512.jpg: 416x416 1 Spanner, Done. (0.007s)
Speed: 0.3ms pre-process, 6.6ms inference, 1.3ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp3
